<a href="https://colab.research.google.com/github/sagihaider/Biogas_Prediction/blob/master/main_Biogas_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/Biogas_Prediction.git

Cloning into 'Biogas_Prediction'...


In [3]:
!pip install xlrd

You should consider upgrading via the 'c:\users\sauga\google drive\packages\research\scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
import numpy as np
import pandas as pd

df=pd.read_excel('/content/Biogas_Prediction/Data/Gasification Data_Daya.xlsx', index_col=None, header=None)
df1 = df.iloc[1:223,1:15] 
print(df1)


print(df1.isnull().values.any())
print(df1.isna().values.any())

# Drop null balues and store dataframe in dataframe 2
df2=df1.dropna()
print(df2)

#Check Null values again after removing
print(df2.isnull().values.any())
print(df2.isna().values.any())

# Store Dependent and independent variabels in X and y
X= df2.iloc[0:187,0:9]
y = df2.iloc[0:187,9:14]

# Convert X and Y in Numpy Array
X=X.to_numpy()
y=y.to_numpy()


FileNotFoundError: [Errno 2] No such file or directory: '/content/Biogas_Prediction/Data/Gasification Data_Daya.xlsx'

In [3]:
from numpy import asarray
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32')

n_features=1
X = X.reshape((X.shape[0], X.shape[1], n_features))


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
n_inputs, n_outputs = X.shape[1], y.shape[1]

In [5]:
# get the model
def get_model(n_inputs, n_outputs):
  # model = Sequential()
  # model.add(Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(n_inputs, n_features)))
  # model.add(Conv1D(filters=8, kernel_size=2, activation='relu'))
  # model.add(Dropout(0.5))
  # model.add(MaxPooling1D(pool_size=2))
  # model.add(Flatten())
  # model.add(Dense(100, activation='relu'))
  # model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
  # model.compile(loss='mae', optimizer='adam')
  # model.summary()
  # return model

  # define model
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_inputs, n_features)))
  model.add(MaxPooling1D(pool_size=2))
  # model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(50, activation='relu'))
  model.add(Dense(n_outputs))
  model.compile(optimizer='adam', loss='mse')
  model.summary()
  return model

In [6]:
model = get_model(n_inputs, n_outputs)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 8, 64)             192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                12850     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 13,297
Trainable params: 13,297
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(X_train, y_train, 
                    epochs=1000, 
                    batch_size=32, 
                    validation_data=(X_test, y_test), verbose=0)

In [8]:
yhat = model.predict(X_test)
# mae = model.evaluate(X_test, y_test, verbose=0)

In [9]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, yhat, multioutput='raw_values')

array([36.237057  , 27.660988  , 32.199512  , 11.443638  ,  0.79450524],
      dtype=float32)